In [1]:
# install toransformer 4.12.5
!pip3 uninstall -y transformers
!pip3 install transformers==4.12.5
!pip3 install sentencepiece

Found existing installation: transformers 4.12.5
Uninstalling transformers-4.12.5:
  Successfully uninstalled transformers-4.12.5
  Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)


In [2]:
# From: https://www.kaggle.com/code/guanghan/deberta-v3-large-baseline-deepshare

# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

# transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")
transformers_path = Path("/usr/local/lib/python3.8/dist-packages/transformers")

# inputの参照
input_dir = Path("../input/deberta-v2-3-fast-tokenizer")

# tokenizerの変更する
convert_file = input_dir / "convert_slow_tokenizer.py"
# .nameは末尾のファイル名(この場合convert_slow_tokenizer)
conversion_path = transformers_path/convert_file.name


if conversion_path.exists():
    conversion_path.unlink()

# convert_fileをtransformers_pathにコピーする
shutil.copy(convert_file, transformers_path)
# debertaの存在するパス
deberta_v2_path = transformers_path / "models" / "deberta_v2"

# 既存のトークナイザを削除して、新しいトークナイザにする
for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

# CFG

In [3]:
# ====================================================
# CFG
# ====================================================
# 後で数字はコメントする
class CFG:
    num_workers=4
    path="../input/deberta-med-score-1"
    config_path=path+'/config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    max_len=354
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

# Library

In [4]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# os.system('pip uninstall -y transformers')
# os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.10.3
transformers.__version__: 4.12.5
env: TOKENIZERS_PARALLELISM=true


# tokenizer

In [5]:
# ====================================================
# tokenizer
# ====================================================
# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

    
from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast
tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-v2-fast-tokenizer')
CFG.tokenizer = tokenizer

# Helper functions for scoring

In [6]:
# From https://www.kaggle.com/theoviel/evaluation-metric-folds-baseline

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds) # ndarrayの連結
    truths = np.concatenate(truths) # ndarrayの連結
    return f1_score(truths, preds)


# span内部に文字列内部のそれぞれの文字がいるかどうかバイナリ数列に変換する関数
def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    バイナリ化した予測とGTを比較する。(microf1値)
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [7]:
array = np.arange(18)
print(array)
print(np.asarray(array<=10))
print(np.where(array<=10)[0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
[ True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [8]:
# スコアをとるため、リストを修正する。
# ;区切りにしたlocation_for_create_labelを用いて、GTを数値リスト化したもののリストを返す。
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df) # 空のlocation_for_create_labelを作成する
    # 各行についてロケーションを;区切りにしたリストをlocation_for_create_labelsに代入する
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths

# textと予測から単語の存在する位置にpred(多分featuresに該当する確率)を代入する
def get_char_probs(texts, predictions, tokenizer):
    # 与えられたテキストの各文について、各分の文字列長分の0行列のリストをつくる
    results = [np.zeros(len(t)) for t in texts]
    # 各文章についてのナンバリングと予測
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        # テキストをトークナイザでidに変換
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True) # return_offsets_mapping: 各単語のオフセット(位置)を取得するか否か
        # 文字の単語位置と予測に対して操作
        # 各文章に対して存在する箇所にpred(多分条件に該当する確率)を入れる
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results

# プローブから、回答の形に整形し返す
def get_results(char_probs, th=0.5):
    results = []
    # 各文章のfeatureに該当している確率について、整形している。
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1 # thを超えた要素のインデックスに1を加えてリザルトに代入。[0]は出てくる2要素目が空なので結果だけ取り出すためのもの
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))] # wakarann
        result = [f"{min(r)} {max(r)}" for r in result] # おそらく閾値thを超えたインデックスが各feature,caseについてリザルトに入っていて、それを返している
        result = ";".join(result)
        results.append(result)
    return results

# ;区切りの結果について、数値リストのリストに変形して出力
def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

# Utils

In [9]:
# ====================================================
# Utils
# ====================================================

# 予測とGTを比較して返す
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score

# なんかログ出すやつ
def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

# seed値指定
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

In [10]:
# ====================================================
# oof
# ====================================================

# oofに対して、最も高い値を出すthを求める

# 事前学習の各foldについての予測値を盾に結合
oof_0 = pd.read_pickle('../input/deberta-med-score-1'+'/oof_df_0.pkl')
oof_1 = pd.read_pickle('../input/deberta-med-score-1'+'/oof_df_1.pkl')
oof_2 = pd.read_pickle('../input/deberta-med-score-1'+'/oof_df_2.pkl')
oof_3 = pd.read_pickle('../input/deberta-med-score-1'+'/oof_df_3.pkl')
oof_4 = pd.read_pickle('../input/deberta-med-score-1'+'/oof_df_4.pkl')


oof = pd.concat([oof_0, oof_1, oof_2, oof_3, oof_4]).reset_index()
#oof.drop_duplicates(inplace=True)

# oofのGTの数値リストのリスト
truths = create_labels_for_scoring(oof)
# CFG.max_len以内の各行について、pred(多分featuresに該当する確率)を代入する。oof[n]にn単語目のthが入っている
char_probs = get_char_probs(oof['pn_history'].values,
                            oof[[i for i in range(CFG.max_len)]].values, 
                            CFG.tokenizer)
best_th = 0.5
best_score = 0.
for th in np.arange(0.45, 0.55, 0.01):
    th = np.round(th, 2)
    results = get_results(char_probs, th=th)
    preds = get_predictions(results)
    score = get_score(preds, truths)
    if best_score < score:
        best_th = th
        best_score = score
    LOGGER.info(f"th: {th}  score: {score:.5f}")
LOGGER.info(f"best_th: {best_th}  score: {best_score:.5f}")

th: 0.45  score: 0.87878
th: 0.46  score: 0.87882
th: 0.47  score: 0.87875
th: 0.48  score: 0.87865
th: 0.49  score: 0.87856
th: 0.5  score: 0.87855
th: 0.51  score: 0.87837
th: 0.52  score: 0.87812
th: 0.53  score: 0.87774
th: 0.54  score: 0.87767
th: 0.55  score: 0.87763
best_th: 0.46  score: 0.87882


# Data Loading

In [11]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
submission = pd.read_csv('../input/nbme-score-clinical-patient-notes/sample_submission.csv')
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"test.shape: {test.shape}")
display(test.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

test.shape: (5, 4)


,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [12]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

,id,case_num,pn_num,feature_num,feature_text,pn_history
0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,Lightheaded,HPI: 17yo M presents with palpitations. Patien...


# Dataset

In [13]:
# ====================================================
# Dataset
# ====================================================

# inputをtorch用に成型するための関数(文章をtorch.Tensorに変形)
def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

# test dataset用のクラス
class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        return inputs

# Model

In [14]:
# モデルの作成
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        # ネットワークの何割をドロップアウトするか定義している
        self.fc_dropout_0 = nn.Dropout(0.1)
        self.fc_dropout_1 = nn.Dropout(0.2)
        self.fc_dropout_2 = nn.Dropout(0.3)
        self.fc_dropout_3 = nn.Dropout(0.4)
        self.fc_dropout_4 = nn.Dropout(0.5)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    # 重みの初期化
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    # inputsを展開してモデルでinferense
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    # ドネットワークの各要素を一定の割合ロップアウトしたネットワークで各アウトプットを足して5で割る
    def forward(self, inputs):
        feature = self.feature(inputs)
        output_0 = self.fc(self.fc_dropout_0(feature))
        output_1 = self.fc(self.fc_dropout_1(feature))
        output_2 = self.fc(self.fc_dropout_2(feature))
        output_3 = self.fc(self.fc_dropout_3(feature))
        output_4 = self.fc(self.fc_dropout_4(feature))
        output = (output_0 + output_1 + output_2 + output_3 + output_4) / 5
        return output

# inference

In [15]:
# ====================================================
# inference
# ====================================================

# モデルにテストデータを入れてインファレンス
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [16]:
# testデータを格納
test_dataset = TestDataset(CFG, test)
# testデータのデータローダーを作成
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path='../input/deberta-med-score-1/config.pth', pretrained=False)

    
    state = torch.load(os.path.join(CFG.path, 'microsoft-deberta-v3-large_fold{}_best.pth'.format(fold)),
                           map_location=torch.device('cpu'))
    
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    prediction = prediction.reshape((len(test), CFG.max_len))
    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()

# 各foldの中で平均とる(各cvはpredictions)
predictions = np.mean(predictions, axis=0)

# print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 266, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThreadException in thread QueueFeederThread:
Traceback (most recent call last):
:
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the a

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
Exception ignored in: <function _ConnectionBase.__del__ at 0x7fd77c244f70>
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
    
During handling of the above exception, another exception occurred:

Tr

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
        _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    _close(self

  0%|          | 0/1 [00:00<?, ?it/s]

Exception ignored in: <function _ConnectionBase.__del__ at 0x7fd77c244f70>
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


# Submission

In [17]:
results = get_results(predictions, th=best_th)
submission['location'] = results
display(submission.head())
submission[['id', 'location']].to_csv('submission.csv', index=False)

,id,location
0,00016_000,696 724
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91;176 183
4,00016_004,222 258


In [26]:
display(test)

,id,case_num,pn_num,feature_num,feature_text,pn_history
0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [28]:
# idを分割してpn_numを抜き出し、pn_history列を加えたデータフレームmodifedを返す関数
submission = pd.read_csv('submission.csv')
display(submission)

def modify_df(df):
    modified_df = df
    id_nums = modified_df['id']
    pn_nums = []
    pns = []
    patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
    
    for id_num in id_nums:
        pn_num, _ = id_num.split('_')
        pn_num = pn_num.lstrip('0')
        pn_nums.append(int(pn_num))
    
    modified_df['pn_num'] = pn_nums
    modified_df = modified_df.merge(patient_notes, on=['pn_num'], how='left')
    
    return modified_df


# location[0]-1を調べてスペース等特殊な文字でなければlocation[0]をデクリメントしてdfを返す関数
def decrement(df):
    locses = df['int_ls_location']
    pn_history = df['pn_history']
    for idx, locs in enumerate(locses):
        for jdx, loc in enumerate(locs):
            loc0 = loc[0]-1
            # 例外処理
            if loc0 < 0:
                continue
            elif pn_history[idx][loc0] in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\newline"]:
                continue
            elif loc0 == 0 and (pn_history[idx][loc0] not in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\newline"]):
                locs[jdx][0] = 0
                continue
            while pn_history[idx][loc0-1] not in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\newline"]:
                loc0 = loc0 - 1
                if loc0 == 0: 
                    break
            locs[jdx][0] = loc0 + 1
        locses[idx] = locs
    return df

# location[1]を調べてスペースでなければlocation[1]をインクリメントする関数
def increment(df):
    locses = df['int_ls_location']
    pn_history = df['pn_history']
    for idx, locs in enumerate(locses):
        for jdx, loc in enumerate(locs):
            loc1 = loc[1]-1
            longest = int(len(pn_history[idx]))
            # 例外処理
            if pn_history[idx][loc1] in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\t"]:
                locs[jdx][1] = loc1
                continue
            elif loc1 == longest-1 and (pn_history[idx][loc0] not in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\t"]):
                locs[jdx][1] = loc1 + 1
                continue
            while pn_history[idx][loc1+1] not in [".", "\"", "\'", ",", ".", ":", " ", "\n", "\r\n", "\t"]:
                loc1 = loc1 + 1
                if loc1 == longest-1: 
                    break
            locs[jdx][1] = loc1 + 1
        locses[idx] = locs
    return df

def make_sub_df(df):
    results = []
    int_locses = df['int_ls_location']
    # int_ls_locationをlocationに整形する
    for int_locs in int_locses:
        result = [f"{loc[0]} {loc[1]}"for loc in int_locs]
        result = ";".join(result)
        results.append(result)
    df["location"] = pd.Series(results)
    df = df.drop(["pn_num", "case_num", "pn_history", "int_ls_location"], axis=1)
    return df

,id,location
0,00016_000,696 725
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91;176 183
4,00016_004,222 258


In [19]:
modified_df = modify_df(submission)
# locationを;で分割したリストと、分割した文字列を" "で分割したリスト行を作る
modified_df['int_ls_location'] = get_predictions(modified_df['location'])
modified_df = decrement(modified_df)
modified_df = increment(modified_df)
submission = make_sub_df(modified_df)
display(submission.head())

,id,location
0,00016_000,696 725
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91;176 183
4,00016_004,222 258


In [20]:
submission[['id', 'location']].to_csv('submission.csv', index=False)

In [29]:
submission = pd.read_csv('submission.csv')
submission['id'] = '00046_009'
submission['location'] = pd.Series(['146 158;436 448;478 491','146 158;436 448;478 484','146 158;436 448;478 484','146 158;436 448;478 484','146 158;436 448;478 484'])

display(submission)
modified_df = modify_df(submission)
modified_df['int_ls_location'] = get_predictions(modified_df['location'])
mdf = decrement(modified_df)
mdf = increment(mdf)
mdf = make_sub_df(mdf)
display(mdf)

,id,location
0,00046_009,146 158;436 448;478 491
1,00046_009,146 158;436 448;478 484
2,00046_009,146 158;436 448;478 484
3,00046_009,146 158;436 448;478 484
4,00046_009,146 158;436 448;478 484


,id,location
0,00046_009,146 158;436 448;478 490
1,00046_009,146 158;436 448;478 490
2,00046_009,146 158;436 448;478 490
3,00046_009,146 158;436 448;478 490
4,00046_009,146 158;436 448;478 490


In [22]:
submission = pd.read_csv('submission.csv')
display(submission)
modified_df = modify_df(submission)
modified_df['int_ls_location'] = get_predictions(modified_df['location'])
mdf = decrement(modified_df)
mdf = increment(mdf)
display(mdf)
print(mdf['pn_history'][1])

,id,location
0,00016_000,696 725
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91;176 183
4,00016_004,222 258


,id,location,pn_num,case_num,pn_history,int_ls_location
0,00016_000,696 725,16,0,HPI: 17yo M presents with palpitations. Patien...,"[[696, 725]]"
1,00016_001,668 693,16,0,HPI: 17yo M presents with palpitations. Patien...,"[[668, 693]]"
2,00016_002,203 217,16,0,HPI: 17yo M presents with palpitations. Patien...,"[[203, 217]]"
3,00016_003,70 91;176 183,16,0,HPI: 17yo M presents with palpitations. Patien...,"[[70, 91], [176, 183]]"
4,00016_004,222 258,16,0,HPI: 17yo M presents with palpitations. Patien...,"[[222, 258]]"


HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms


In [23]:
print(modified_df)

          id       location  pn_num  case_num                                         pn_history         int_ls_location
0  00016_000        696 725      16         0  HPI: 17yo M presents with palpitations. Patien...            [[696, 725]]
1  00016_001        668 693      16         0  HPI: 17yo M presents with palpitations. Patien...            [[668, 693]]
2  00016_002        203 217      16         0  HPI: 17yo M presents with palpitations. Patien...            [[203, 217]]
3  00016_003  70 91;176 183      16         0  HPI: 17yo M presents with palpitations. Patien...  [[70, 91], [176, 183]]
4  00016_004        222 258      16         0  HPI: 17yo M presents with palpitations. Patien...            [[222, 258]]


In [ ]:
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
display(patient_notes[patient_notes['pn_num']==16])
display(train)

In [25]:
submission = pd.read_csv('submission.csv')
display(submission)

,id,location
0,00016_000,696 725
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91;176 183
4,00016_004,222 258
